In [15]:
%env JAX_PLATFORM_NAME=cpu

import jaxtyping  # noqa: F401

%load_ext jaxtyping
%jaxtyping.typechecker beartype.beartype

env: JAX_PLATFORM_NAME=cpu
The jaxtyping extension is already loaded. To reload it, use:
  %reload_ext jaxtyping


In [16]:
import jax
import jax.numpy as jnp
import equinox as eqx
import optax

from tqdm.notebook import trange

from jaxtyping import Array, Bool, Float

from chaogatenn.chaogate import ChaoGate
from chaogatenn.maps import LogisticMap
from chaogatenn.utils import grad_norm

In [17]:
# Training data for the AND gate
X = jnp.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=bool)  # Input combinations
AND_Y = jnp.array([0, 0, 0, 1], dtype=bool)  # AND gate output
OR_Y = jnp.array([0, 1, 1, 1], dtype=bool)  # OR gate output
XOR_Y = jnp.array([0, 1, 1, 0], dtype=bool)  # XOR
NAND_Y = jnp.array([1, 1, 1, 0], dtype=bool)  # NAND

Y = OR_Y

In [18]:
Map = LogisticMap(a=1.0)
# Map = LorenzMap()
# Map = DuffingMap(
#     alpha=1.0, beta=5.0, delta=0.02, gamma=8.0, omega=0.5, dt=0.01, steps=1000
# )

In [19]:
Map(2)

-2.0

In [20]:
chao_gate = ChaoGate(DELTA=0.0, X0=0.0, X_THRESHOLD=1.0, Map=Map)

In [21]:
[chao_gate(x) for x in X]

[Array(0.26894143, dtype=float32, weak_type=True),
 Array(0.26894143, dtype=float32, weak_type=True),
 Array(0.26894143, dtype=float32, weak_type=True),
 Array(0.26894143, dtype=float32, weak_type=True)]

In [22]:
@eqx.filter_value_and_grad()
def compute_loss(
    chao_gate: ChaoGate, x: Bool[Array, "batch 2"], y: Bool[Array, "batch"]
) -> Float[Array, ""]:  # noqa: F821
    pred = jax.vmap(chao_gate)(x)
    # binary cross entropy
    return -jnp.mean(y * jnp.log(pred + 1e-15) + (1 - y) * jnp.log(1 - pred + 1e-15))

In [23]:
@eqx.filter_jit
def make_step(
    model: ChaoGate,
    x: Bool[Array, "dim 2"],
    y: Bool[Array, "dim"],  # noqa: F821
    optim: optax.GradientTransformation,
    opt_state: optax.OptState,
) -> (Float[Array, "dim"], ChaoGate, optax.OptState):  # type: ignore  # noqa: F821
    loss, grads = compute_loss(model, x, y)
    updates, opt_state = optim.update(grads, opt_state)
    # jax.debug.print(f"{grads, updates}")
    model = eqx.apply_updates(model, updates)
    return loss, model, opt_state

In [24]:
optim = optax.adabelief(3e-4)
opt_state = optim.init(eqx.filter(chao_gate, eqx.is_inexact_array))

In [25]:
epochs = 4_000

for epoch in trange(epochs):
    loss, chao_gate, opt_state = make_step(chao_gate, X, Y, optim, opt_state)  # type: ignore
    _, grads = compute_loss(chao_gate, X, Y)
    grad_norm_value = grad_norm(grads)
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss}, Grad Norm: {grad_norm_value}")

  0%|          | 0/4000 [00:00<?, ?it/s]

Epoch 0, Loss: 1.06326162815094, Grad Norm: 0.7178342938423157
Epoch 10, Loss: 1.0588704347610474, Grad Norm: 0.7095215916633606
Epoch 20, Loss: 1.0527642965316772, Grad Norm: 0.6982076168060303
Epoch 30, Loss: 1.045273780822754, Grad Norm: 0.6846041679382324
Epoch 40, Loss: 1.0367717742919922, Grad Norm: 0.6694703698158264
Epoch 50, Loss: 1.0275639295578003, Grad Norm: 0.6534243226051331
Epoch 60, Loss: 1.0178829431533813, Grad Norm: 0.6369349360466003
Epoch 70, Loss: 1.0079035758972168, Grad Norm: 0.6203510761260986
Epoch 80, Loss: 0.9977566599845886, Grad Norm: 0.603927731513977
Epoch 90, Loss: 0.9875398874282837, Grad Norm: 0.587847888469696
Epoch 100, Loss: 0.9773253202438354, Grad Norm: 0.572238564491272
Epoch 110, Loss: 0.967165470123291, Grad Norm: 0.5571839213371277
Epoch 120, Loss: 0.9570975303649902, Grad Norm: 0.5427358150482178
Epoch 130, Loss: 0.9471473097801208, Grad Norm: 0.5289220213890076
Epoch 140, Loss: 0.9373314380645752, Grad Norm: 0.515752375125885
Epoch 150, Los

In [26]:
print("\nTrained ChaoGate Parameters:")
print(
    f"DELTA: {chao_gate.DELTA}, X0: {chao_gate.X0}, X_THRESHOLD: {chao_gate.X_THRESHOLD}"
)


Trained ChaoGate Parameters:
DELTA: 2.0704758167266846, X0: -0.5012881755828857, X_THRESHOLD: -4.73203182220459


In [27]:
[
    (
        bool(x1.item()),
        bool(x2.item()),
        (
            chao_gate.Map(chao_gate.X0 + x1 * chao_gate.DELTA + x2 * chao_gate.DELTA)
            > chao_gate.X_THRESHOLD
        ).item(),
    )
    for x1, x2 in X
]

[(False, False, True),
 (False, True, True),
 (True, False, True),
 (True, True, False)]

In [28]:
pred_ys = jax.vmap(chao_gate)(X)
num_correct = jnp.sum((pred_ys > 0.5) == Y)
final_accuracy = (num_correct / len(X)).item()
print(f"final_accuracy={final_accuracy}")

final_accuracy=1.0
